### Sales Prediction

**Motivation! What's the context?**
- In 2017 Olist started to forecast its monthly sales based on the average revenue from 3 months before. But now the company  wants to be able to predict future sales more accurately.

**Why?**
- To efficiently allocate resources for future projects, growth and manage its cash flow. 

**Who are the stakeholders?**
- The financial sector will use the forecast to the set operating budgets and to project cash flows.
- And the marketing team, who will have the opportunity to schedule promotions if it appears sales <br>
will be weak and will be able to better allocate budget among other marketing activities.

**What is the deliverable?**
- A dashboard with the information about the forecast.


> **ps: We will use the data to forecast the second quarter of 2018 and the third quarter will be our "production data".**

In [ ]:
df['order_purchase_timestamp_month_sin'] = df['order_purchase_timestamp_month'].apply( lambda x: np.sin( x * ( 2. * np.pi/12 ) ) )
df['order_purchase_timestamp_month_cos'] = df['order_purchase_timestamp_month'].apply( lambda x: np.cos( x * ( 2. * np.pi/12 ) ) )

df['order_estimated_delivery_date_month_sin'] = df['order_estimated_delivery_date_month'].apply( lambda x: np.sin( x * ( 2. * np.pi/12 ) ) )
df['order_estimated_delivery_date_month_cos'] = df['order_estimated_delivery_date_month'].apply( lambda x: np.cos( x * ( 2. * np.pi/12 ) ) )

df['order_purchase_timestamp_day_sin'] = df['order_purchase_timestamp_day'].apply( lambda x: np.sin( x * ( 2. * np.pi/30 ) ) )
df['order_purchase_timestamp_day_cos'] = df['order_purchase_timestamp_day'].apply( lambda x: np.cos( x * ( 2. * np.pi/30 ) ) )


df['order_purchase_timestamp_weekofyear_sin'] = df['order_purchase_timestamp_weekofyear'].apply( lambda x: np.sin( x * ( 2. * np.pi/52 ) ) )
df['order_purchase_timestamp_weekofyear_cos'] = df['order_purchase_timestamp_weekofyear'].apply( lambda x: np.cos( x * ( 2. * np.pi/52 ) ) )

df['order_estimated_delivery_date_weekofyear_sin'] = df['order_estimated_delivery_date_weekofyear'].apply( lambda x: np.sin( x * ( 2. * np.pi/52 ) ) )
df['order_estimated_delivery_date_weekofyear_cos'] = df['order_estimated_delivery_date_weekofyear'].apply( lambda x: np.cos( x * ( 2. * np.pi/52 ) ) )

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_city = LabelEncoder()
le_state = LabelEncoder()
df['customer_city'] = le_city.fit_transform( df['customer_city'] )
df['customer_state'] = le_state.fit_transform( df['customer_state'] )

In [ ]:
mask = (df['order_purchase_timestamp'] < '2018-4-1')

x_train = df.loc[mask].drop(columns=['price'])
y_train = df.loc[mask, 'price']


mask = (df['order_purchase_timestamp'] >= '2018-4-1')
x_test = df.loc[mask].drop(columns=['price'])
y_test = df.loc[mask, 'price']

In [ ]:
to_drop = ['order_purchase_timestamp', 'order_estimated_delivery_date']
x_train.drop(columns=to_drop, inplace=True)
x_test.drop(columns=to_drop, inplace=True)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
ex = ExtraTreesRegressor(min_samples_leaf=9)
ex.fit(x_train, y_train)

In [ ]:
predictions = ex.predict(x_test)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(y_test, predictions)

In [ ]:
mask = (df['order_purchase_timestamp'] >= '2018-4-1')

In [ ]:
a = pd.concat([df.loc[mask]['order_purchase_timestamp'], y_test], axis=1)
a = pd.concat([a.reset_index(drop=True), pd.Series(predictions, name='p')], axis=1)

In [ ]:
a['purchase_date'] = a['order_purchase_timestamp'].dt.date
a.groupby('purchase_date').sum().plot(figsize=(15,5))

In [ ]:
a.groupby(a['order_purchase_timestamp'].dt.month).sum().plot(figsize=(15,5))